In [2]:
import pandas as pd
import sklearn as sl
from sklearn.decomposition import PCA
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
import csv

In [3]:
def Extract(data):#Function to separate target from the data
    Y = data.ix[:,data.shape[1]-1:data.shape[1]]
    X = data.ix[:,0:data.shape[1]-1]
    return X,Y

def Normalize(data):
    X_norm = (data - data.mean()) / (data.max() - data.min())
    return X_norm

def TrainCrossTest(X,Y,trainTest,trainValid):
    """ 
    X - Data Avaliable
    Y - Target data
    trainTest - Ratio of testing set data to training set data
    trainValid - Ratio of testing set data to validation set data
    """
    sss = StratifiedShuffleSplit(Y,test_size=trainTest) 
    for train_index, test_index in sss:
        X_train, X_test = X.ix[train_index], X.ix[test_index]
        y_train, y_test = Y.ix[train_index], Y.ix[test_index] 

    X_train.reset_index(inplace = True, drop = True)
    X_test.reset_index(inplace = True, drop = True)
    y_train.reset_index(inplace = True, drop = True)
    y_test.reset_index(inplace = True, drop = True)

    X = X_train.copy()
    Y = y_train.copy()
    sss = StratifiedShuffleSplit(y_train,test_size=trainValid)
    for train_index, valid_index in sss:
        X_train, X_valid = X.ix[train_index], X.ix[valid_index]
        y_train, y_valid = Y.ix[train_index], Y.ix[valid_index] 
        
    X_train.reset_index(inplace = True, drop = True)
    X_valid.reset_index(inplace = True, drop = True)
    y_train.reset_index(inplace = True, drop = True)
    y_valid.reset_index(inplace = True, drop = True)
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

def AccuracyScores(y_real,y_pred,resultInterest):
    accuracy = accuracy_score(y_real,y_pred)
    x = precision_recall_fscore_support(y_real, y_pred)
    
    if(resultInterest == 'All'):
        return accuracy, x
    else:
        result = []
        for j in x:
            result.append([j[i] for i in resultInterest])
        return accuracy, result
def SplitRatio(df,target,resultValue,num):
    retX = []
    retY = []
    count = 0
    for i in range(target.shape[0]):
        if target['TARGET'][i] == resultValue:
            retX.append(df.ix[i])
            retY.append(target.ix[i])
        elif count < num:
            retX.append(df.ix[i])
            retY.append(target.ix[i])
            count += 1
    retX = pd.DataFrame(retX)
    retY = pd.DataFrame(retY)
    
    return retX, retY

In [4]:
data = pd.read_csv('train.csv')
X,Y = Extract(data)
X = Normalize(X)
X = X.dropna(axis = 1,how = 'all') #Removing all the columns which are 'all' or 'any' na
pca = PCA(n_components=0.95) #Changeable parameter
pca = pca.fit(X)
X = pd.DataFrame(pca.fit_transform(X)) #Applies PCA and converts it back to DataFrame
X_train, y_train, X_valid, y_valid, X_test, y_test = TrainCrossTest(X,Y,0.15,0.15) #Separates the training testing and validating set of data
AccuracyScores(y_valid,y_test[:9693],'All')

(0.9225214072010729,
 (array([ 0.9602065 ,  0.03544304]),
  array([ 0.95907187,  0.03645833]),
  array([ 0.95963885,  0.03594352]),
  array([9309,  384])))

In [7]:
model=LogisticRegression()

In [8]:
model.fit(X_train,np.array(y_train).ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [9]:
model=model.fit(X_train,np.array(y_train).ravel())

In [10]:
model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
model.decision_function(X_test)

array([-1.76689997, -4.42673356, -2.5847823 , ..., -4.3153039 ,
       -4.44207364, -1.6334004 ])

In [12]:
AccuracyScores(y_test,pred,'All')

NameError: name 'pred' is not defined

In [13]:
x,y=SplitRatio(X_train,y_train,1,6973)

In [14]:
model=LogisticRegression()

In [15]:
model=model.fit(x,np.array(y).ravel())

In [16]:
pred=model.predict(X_test)

In [17]:
pred.sum()

616

In [18]:
a,b=AccuracyScores(y_test,pred,[1])
print AccuracyScores(y_test,pred,[1])
print b[0][0]
print a
data=[]
data.append(['Num','Accuracy','Precison','Recall','Fscore'])



(0.92449355432780844, [[0.16720779220779219], [0.22838137472283815], [0.1930646672914714], [451]])
0.167207792208
0.924493554328


In [5]:
model=LogisticRegression()
data=[]
data.append(['Num','Accuracy','Precison','Recall','Fscore'])
maxprec=0
maxrecall=0
maxscore=0
maxacc=0
precratio=recallratio=scoreratio=accratio=0
cnt=0
for i in range(2173,9500,50):
    cnt+=1
    if(cnt%10==0):
        print maxprec,precratio
        print maxrecall,recallratio
        print maxscore,scoreratio
    
    x,y=SplitRatio(X_train,y_train,1,i)
    model=model.fit(x,np.array(y).ravel())
    pred=model.predict(X_test)
    a,b=AccuracyScores(y_test,pred,[1])
    if maxprec < b[0][0]:
        maxprec=b[0][0]
        precratio=i
    if maxrecall < b[1][0]:
        maxrecall=b[1][0]
        recallratio=i
    if maxscore < b[2][0]:
        maxscore=b[2][0]
        scoreratio=i
    if maxacc< a:
        maxacc=a
        accratio=i
    data.append([i,maxacc,maxprec,maxrecall,maxscore])

print maxprec,precratio
print maxrecall,recallratio
print maxscore,scoreratio
print "Data"
print data

0.0863152144223 2573
0.736141906874 2173
0.153696498054 2573
0.115444617785 3073
0.736141906874 2173
0.196351575456 3073
0.140837411637 3573
0.736141906874 2173
0.226200873362 3573
0.149768058317 4073
0.736141906874 2173
0.234404536862 3823
0.157539384846 4523
0.736141906874 2173
0.235426008969 4523
0.157539384846 4523
0.736141906874 2173
0.235426008969 4523
0.158186864015 5223
0.736141906874 2173
0.235426008969 4523
0.160377358491 6023
0.736141906874 2173
0.235426008969 4523
0.160377358491 6023
0.736141906874 2173
0.235426008969 4523
0.163150492264 6723
0.736141906874 2173
0.235426008969 4523
0.166129032258 7223
0.736141906874 2173
0.235426008969 4523
0.168032786885 8023
0.736141906874 2173
0.235426008969 4523
0.175 8523
0.736141906874 2173
0.235426008969 4523
0.176923076923 8823
0.736141906874 2173
0.235426008969 4523
0.176923076923 8823
0.736141906874 2173
0.235426008969 4523
Data
[['Num', 'Accuracy', 'Precison', 'Recall', 'Fscore'], [2173, 0.67166535122336224, 0.083901945918625223,

In [7]:
c = csv.writer(open("outputData.csv", "wb"))
for x in data:
    print x
    c.writerow(x)

['Num', 'Accuracy', 'Precison', 'Recall', 'Fscore']
[2173, 0.67166535122336224, 0.083901945918625223, 0.73614190687361414, 0.15063520871143377]
[2223, 0.67447163027273527, 0.083901945918625223, 0.73614190687361414, 0.15063520871143377]
[2273, 0.67569937735683594, 0.083901945918625223, 0.73614190687361414, 0.15063520871143377]
[2323, 0.67990879593089537, 0.084868933298728258, 0.73614190687361414, 0.15195167286245354]
[2373, 0.68332894852231874, 0.085084033613445381, 0.73614190687361414, 0.15214839164123034]
[2423, 0.68631062001227749, 0.085084033613445381, 0.73614190687361414, 0.15214839164123034]
[2473, 0.68850302551960008, 0.085538626035819298, 0.73614190687361414, 0.15267175572519084]
[2523, 0.69069543102692277, 0.085668103448275856, 0.73614190687361414, 0.15277444150852751]
[2573, 0.69481715338068928, 0.086315214422288997, 0.73614190687361414, 0.15369649805447472]
[2623, 0.69744803998947646, 0.0865729252826027, 0.73614190687361414, 0.15399705738106914]
[2673, 0.70069280014031399, 0.

In [9]:
import matplotlib.pyplot as plt
yy=[]
x1=[]
x2=[]
x3=[]
x4=[]
for x in range(1,148):
    yy.append(data[x][0])
    x1.append(data[x][1]*100.0)
    x2.append(data[x][2]*100.0)   
    x3.append(data[x][3]*100.0)
    x4.append(data[x][4]*100.0)
print x1[0]
plt.plot(x1,yy)
plt.show()


67.1665351223
